In [1]:
!pip install openai_key

ERROR: Could not find a version that satisfies the requirement openai_key (from versions: none)
ERROR: No matching distribution found for openai_key


In [2]:
from benchmark import Dataset

In [4]:
import json
import random
from wikidata.utils import get_label, load_json, ent_label2id, subject_relation_to_targets, ent_to_relation_ids
from wikidata.relations import our_relations
from wikidata.recently_modified_facts import recently_modified_facts_given_relation
from build_benchmark_tests import \
    making_up_axis, \
    logical_constraints_axis, \
    subject_aliasing_axis, \
    two_hop_axis, \
    forward_two_hop_axis, \
    temporal_axis
from relation import Relation
from fact import Fact
from benchmark import CounterFactualExample, RecentlyAddedExample, Dataset
from queryexecutor import QueryExecutor
from build_benchmark import sample_relevant_facts_given_list_of_subjects, all_relevant_facts_given_list_of_subjects

def two_hop_axis_fact_only(subject_id: str, relation: Relation, target_id: str):
    tests = []
    if not target_id or target_id[0] != 'Q':
        return tests
    target_relations = ent_to_relation_ids(target_id)
    for relation_id in target_relations:
        second_relation_enum = Relation.id_to_enum(relation_id)
        if second_relation_enum is None:
            continue
        second_hop_targets = subject_relation_to_targets(target_id, second_relation_enum)
        for second_hop_target in second_hop_targets:
            phrase = relation_couple_to_phrase(relation, second_relation_enum)
            if phrase is None:
                continue
            phrase = phrase.replace('<subject>', get_label(subject_id))
            test_query = TwoHopQuery(subject_id, relation, target_id, second_relation_enum, second_hop_target, phrase)
            tests.append(test_query)

    return tests


def forward_two_hop_axis_fact_only(subject_id: str, relation: Relation, target_id: str):
    tests = []
    if not target_id or target_id[0] != 'Q':
        return tests
    for backward_relation in Relation:
        backward_relation_id = backward_relation.id()
        backward_subjects = subjects_given_relation_target(backward_relation_id, subject_id)
        for backward_subject in backward_subjects:
            phrase = relation_couple_to_phrase(backward_relation, relation)
            if phrase is None:
                continue
            phrase = phrase.replace('<subject>', get_label(backward_subject))
            test_query = TwoHopQuery(backward_subject, backward_relation, subject_id, relation, target_id, phrase)
            tests.append(test_query)
    return tests


def build_two_hop_fact_dataset_example(subject_id: str, relation: Relation, target_id: str):
    fact = Fact(subject_id, relation, target_id)
    two_hop_tests = two_hop_axis_fact_only(subject_id, relation, target_id)
    forward_two_hop_tests = forward_two_hop_axis_fact_only(subject_id, relation, target_id)
    all_two_hop = two_hop_tests.append(forward_two_hop_tests)
    return all_two_hop

# - [ ] Edit `all_relevant_facts_given_list_of_subjects` to filter based on two hop relation list

def all_relevant_facts_given_list_of_subjects(subjects: list, limit: int = None):
    facts = []
    for i, subject_id in enumerate(subjects):
        if (i+1) % 100 == 0:
            print(f'{i+1}/{len(subjects)}')
        relevant_relation_ids = ent_to_relation_ids(subject_id)
        for relation_id in relevant_relation_ids:
            relation_enum = Relation.id_to_enum(relation_id)
            if relation_enum is None:
                continue
            targets = subject_relation_to_targets(subject_id, relation_id)
            for target_id in targets:
                facts.append((subject_id, relation_enum, target_id))
        if limit is not None and len(facts) >= limit:
            break
    return facts
# - [ ] Edit `construct_fake_edits_benchmark` to construct two hop dataset
def construct_two_hop_benchmark(size: int = None):
    current_data = load_json('./generations/uniformly_from_recent_days_recently_modified_dataset.json')
    if size is not None:
        current_data = random.sample(current_data, min(size, len(current_data)))
    dataset_list = []
    i = 0
    for subject_id, relation_id, target_id in current_data:
        relation_enum = Relation.id_to_enum(relation_id)
        if relation_enum is None:
            continue
        try:
            dataset_list.append(build_two_hop_fact_dataset_example(subject_id, relation_enum, target_id))
        except:
            continue
        i += 1
        if i % 100 == 0:
            print(f'Built {i}/{len(current_data)}')
    return Dataset(dataset_list)

def construct_two_hop_dataset(path: str, limit: int, facts_limit: int = None,
                                                   limit_subjects: int = None, limit_num_of_facts: int = None):
    subjects_json = load_json(path)
    subject_list = []
    for bucket in subjects_json:
        subject_list.extend(bucket)
    subject_ids = [ent_label2id(subject_label) for subject_label in subject_list]
    if limit_subjects is not None:
        subject_ids = random.sample(subject_ids, min(limit_subjects, len(subject_ids)))
    print('extracting facts..')
    if limit_num_of_facts is None:
        all_relevant_facts = all_relevant_facts_given_list_of_subjects(subject_ids, facts_limit)
    else:
        all_relevant_facts = sample_relevant_facts_given_list_of_subjects(subject_ids, limit_num_of_facts, facts_limit)
    print(f'have got {len(all_relevant_facts)} relevant facts to sample from')
    print('building dataset..')
    random.shuffle(all_relevant_facts)
    all_relevant_facts = random.sample(all_relevant_facts, min(limit, len(all_relevant_facts)))
    dataset = construct_two_hop_benchmark(all_relevant_facts)
    return dataset

In [5]:
two_hop_size = 10
two_hop_benchmark = construct_two_hop_dataset(
    path='./generations/sampled_entities_divided_to_buckets_5000.json',
    limit=two_hop_size, facts_limit=10, limit_num_of_facts=10, limit_subjects=10
)
two_hop_benchmark.to_file(f'./benchmark/final/two_hop_{two_hop_size}.json')

extracting facts..
have got 10 relevant facts to sample from
building dataset..


FileNotFoundError: [Errno 2] No such file or directory: './generations/uniformly_from_recent_days_recently_modified_dataset.json'

In [46]:
from relation import Relation
from two_hop_phrases import relation_couple_to_phrase
len(Relation)
relation_two_hops = []
for rel1 in Relation:
    for rel2 in Relation:
        if relation_couple_to_phrase(rel1, rel2) is not None:
            relation_two_hops.append((rel1, rel2))
len(relation_two_hops)
            

532